The purpose of this notebook is to post-process a .psv.gz simulation file to add schools and FEMA regions. This does not represent how an operational version would run... it's simply to add these new features to show off on the example page of the v2 dashboard. 

In [67]:
import json
from io import StringIO
import gzip

from pygridder import pygridder as pgrid

In [9]:
import pathlib
import glob
import os

import argparse
import geojson
import pandas as pd
import json
import random
import numpy as np
import pyproj

from datetime import datetime, timedelta

In [3]:
# Read in example psv
file = '../data/output/examples/20210428124600.psv.gz'
df = pd.read_csv(file, sep="|")

In [4]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,time,slon,slat,elon,elat
0,1,1056,2.0662,1,NY,36013,BUF,0,0,0,0,0,0,22,-79.1592,42.4951,-79.1388,42.5209
1,1,480,2.6825,1,NY,36097,BGM,0,0,0,0,0,0,21,-76.9440,42.4153,-76.9021,42.4388
2,1,0,4.5373,2,TX,48023,OUN,0,0,0,0,0,0,2,-99.0422,33.6459,-98.9941,33.6980
3,1,80,0.7748,0,TX,48143,FWD,0,0,0,0,0,0,22,-98.1495,32.0895,-98.1437,32.0995
4,1,368,3.8219,1,TX,"48367,48497",FWD,0,0,0,0,0,0,23,-97.8818,33.0339,-97.8283,33.0662


### Get projection information to properly grid lat/lon lines and get school #'s

In [5]:
ndfd_file = '../../dev/impacts-app/scripts/impacts-data/orig-gis-data/ndfd.npz'

In [36]:
impacts_file = './example_files/impact-grids-new.npz'

In [10]:
with np.load(ndfd_file) as NPZ:
    X = NPZ['X']
    Y = NPZ['Y']
    proj = pyproj.Proj(NPZ['srs'].item())

In [11]:
G = pgrid.Gridder(X,Y)

In [54]:
with np.load(impacts_file) as NPZ:
    schools = NPZ['schools']

- iterate through df rows
- for each row...
    - project lons/lats into ndfd points
    - grid lines using these ndfd start/end points
    - use returned indices to grab and sum school counts for the tornado
    - set that value to a new column 'schools'

Example run through

In [15]:
tor = df.iloc[0,:]

In [17]:
x1s, y1s = proj(tor['slon'],tor['slat'])
x2s, y2s = proj(tor['elon'],tor['elat'])

In [18]:
idxs = G.grid_lines(x1s,y1s,x2s,y2s)

In [43]:
f'Schools "Exposed": {schools[idxs[0]].sum()}'

'Schools "Exposed": 4'

### Processing all tors and getting school counts

In [61]:
school_total = []

for row in np.arange(df.shape[0]):
    
    tor = df.iloc[row,:]
    
    x1s, y1s = proj(tor['slon'],tor['slat'])
    x2s, y2s = proj(tor['elon'],tor['elat'])
    
    idxs = G.grid_lines(x1s,y1s,x2s,y2s)
    
    school_count = schools[idxs[0]].sum()
    school_total.append(school_count)

In [62]:
df['schools'] = school_total

In [69]:
psv_str = df.to_csv(sep='|',index=False)

with gzip.GzipFile('../data/output/examples/20210428124600_schools.psv.gz','w') as OUT:
    OUT.write(psv_str.encode())

Making sure the file is OK

In [70]:
# Read in example psv
file = '../data/output/examples/20210428124600_schools.psv.gz'
df_new = pd.read_csv(file, sep="|")

### Add FEMA Column

In [72]:
fema_dict = {
    'WA': '10',
    'OR': '10',
    'ID': '10',
    'CA': '9', 
    'NV': '9',
    'AZ': '9',
    'MT': '8',
    'WY': '8',
    'UT': '8',
    'CO': '8',
    'ND': '8',
    'SD': '8',
    'NE': '7',
    'KS': '7',
    'IA': '7',
    'MO': '7',
    'NM': '6',
    'TX': '6',
    'OK': '6',
    'AR': '6',
    'LA': '6',
    'MN': '5',
    'WI': '5',
    'MI': '5',
    'IL': '5',
    'IN': '5',
    'OH': '5',
    'KY': '4',
    'TN': '4',
    'MS': '4',
    'AL': '4',
    'GA': '4',
    'FL': '4',
    'SC': '4',
    'NC': '4',
    'VA': '3',
    'WV': '3',
    'PA': '3',
    'DC': '3',
    'MD': '3',
    'DE': '3',
    'NY': '2',
    'NJ': '2',
    'CT': '1',
    'RI': '1',
    'MA': '1',
    'VT': '1',
    'NH': '1',
    'ME': '1'
}

In [104]:
fema = []

for row in np.arange(df.shape[0]):
    
    tor = df.iloc[row,:]
    
    # Offshore tornadoes have 'nan' for the state
    try:
        states = tor['states'].split(',')
    except AttributeError:
        fema.append(np.nan)
        continue
    
    if len(states) > 1:
        
        fema_list = list(map(lambda state: fema_dict[state], states))
        fema_str = ','.join(fema_list)
        fema.append(fema_str)
        
    else:
        fema.append(fema_dict[tor['states']])

In [105]:
df['fema'] = fema

In [107]:
psv_str = df.to_csv(sep='|',index=False)

with gzip.GzipFile('../data/output/examples/20210428124600_schools_fema.psv.gz','w') as OUT:
    OUT.write(psv_str.encode())